In [1]:
import torchaudio
import os
import matplotlib.pyplot as plt



In [2]:
MAIN_FOL = "../data/vox1_dev_dataset"

In [3]:
import librosa
import pandas as pd
from tqdm import tqdm

data = []
NUM_SPK = 1000
NUM_AUD_PER_SPK = 50
for spk_id in tqdm(os.listdir(os.path.join(MAIN_FOL, "wav"))[:NUM_SPK]):
    spk_aud = 0
    for vid_id in os.listdir(os.path.join(MAIN_FOL, "wav", spk_id)):
        for file_id in os.listdir(os.path.join(MAIN_FOL, "wav", spk_id, vid_id)):
            filepath = os.path.join(MAIN_FOL, "wav", spk_id, vid_id, file_id)
            waveform, sample_rate = librosa.load(filepath)
            duration = librosa.get_duration(y=waveform, sr=sample_rate)
            data.append({
                'unique_id': f"{spk_id}_{vid_id}_{file_id}",
                'speaker_id': spk_id,
                'video_id': vid_id,
                'file_id': file_id,
                'filepath': filepath,
                'sample_rate': sample_rate,
                'duration': duration
            })
            
            spk_aud += 1
            if spk_aud >= NUM_AUD_PER_SPK:
                break
        if spk_aud >= NUM_AUD_PER_SPK:
            break
            
            

100%|██████████| 1000/1000 [01:03<00:00, 15.73it/s]


In [4]:
import pandas as pd

df = pd.DataFrame(data)

In [5]:
df

,unique_id,speaker_id,video_id,file_id,filepath,sample_rate,duration
0,id10674_vCISva6imEc_00001.wav,id10674,vCISva6imEc,00001.wav,../data/vox1_dev_dataset/wav/id10674/vCISva6im...,22050,5.960091
1,id10674_VA6tKn0Jgek_00001.wav,id10674,VA6tKn0Jgek,00001.wav,../data/vox1_dev_dataset/wav/id10674/VA6tKn0Jg...,22050,4.200091
2,id10674_VA6tKn0Jgek_00003.wav,id10674,VA6tKn0Jgek,00003.wav,../data/vox1_dev_dataset/wav/id10674/VA6tKn0Jg...,22050,7.040091
3,id10674_VA6tKn0Jgek_00005.wav,id10674,VA6tKn0Jgek,00005.wav,../data/vox1_dev_dataset/wav/id10674/VA6tKn0Jg...,22050,4.880091
4,id10674_VA6tKn0Jgek_00007.wav,id10674,VA6tKn0Jgek,00007.wav,../data/vox1_dev_dataset/wav/id10674/VA6tKn0Jg...,22050,5.600091
...,...,...,...,...,...,...,...
49870,id10141_2djfpVN_86k_00005.wav,id10141,2djfpVN_86k,00005.wav,../data/vox1_dev_dataset/wav/id10141/2djfpVN_8...,22050,4.640091
49871,id10141_2djfpVN_86k_00002.wav,id10141,2djfpVN_86k,00002.wav,../data/vox1_dev_dataset/wav/id10141/2djfpVN_8...,22050,6.400091
49872,id10141_2djfpVN_86k_00004.wav,id10141,2djfpVN_86k,00004.wav,../data/vox1_dev_dataset/wav/id10141/2djfpVN_8...,22050,10.480091
49873,id10141_UOxg8l4ykPQ_00001.wav,id10141,UOxg8l4ykPQ,00001.wav,../data/vox1_dev_dataset/wav/id10141/UOxg8l4yk...,22050,4.400091


In [6]:
stats = df.groupby('speaker_id').agg({'duration':'sum', 'unique_id':'count'}).rename(columns={'duration':'total_duration', 'unique_id':'num_utterances'}).reset_index()

In [7]:
stats['num_by_perce'] = stats["num_utterances"] / stats["num_utterances"].sum()

In [8]:
stats.sort_values(by='num_by_perce', ascending=False)

,speaker_id,total_duration,num_utterances,num_by_perce
0,id10001,363.604535,50,0.001003
680,id10864,610.204535,50,0.001003
651,id10830,353.524535,50,0.001003
652,id10831,317.604535,50,0.001003
653,id10832,415.164535,50,0.001003
...,...,...,...,...
534,id10684,288.244082,45,0.000902
505,id10650,434.844082,45,0.000902
981,id11229,361.044082,45,0.000902
158,id10189,422.084082,45,0.000902


In [9]:
from IPython.display import Audio, display

for i,row in df.iterrows():
    print(row["filepath"])
    waveform, sample_rate = librosa.load(row["filepath"])
    resampled_waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr=6000)
    display(Audio(resampled_waveform[4000:8000], rate=6000))
    display(Audio(waveform[4000:8000], rate=sample_rate))
    print(waveform.shape, resampled_waveform.shape)
    break

../data/vox1_dev_dataset/wav/id10674/vCISva6imEc/00001.wav


(131420,) (35761,)


In [10]:
df.to_csv(os.path.join(MAIN_FOL, "unseen_data.csv"), index=False)